In [ ]:
D:\IntroToDataMining-CS412\final project\benchmark\dataset0

In [ ]:
E = ['A', 'T', 'C', 'G']

In [5]:
import random
import numpy as np
import math
import copy

In [6]:
##### 读.fa
fa_in = open("D:/IntroToDataMining-CS412/final project/benchmark/dataset3/sequences.fa","r")
fa_Seq = []#定义一个列表，用于存储序列
fa_Num = 0
for line in fa_in.readlines():#每次从FASTA文件中读取一行
    line = line.rstrip()#去掉行末的换行符
    fa_Num = fa_Num + 1
    fa_Seq.append(line)
#fa_Seq
##### 读motif 长度
motiflen_in = open("D:/IntroToDataMining-CS412/final project/benchmark/dataset3/motiflength.txt","r")
motif_len = int(motiflen_in.readline())
#motif_len

In [7]:
def generate_prob(pro_matrix,k_mer): #k_mer 'CAAATCCC'
    '''
    给定pro_matrix的情况下，出现当前k_mer的概率
    '''
    dic = {'A':0,'T':1,'C':2,'G':3}
    p = 1
    for i in range(len(k_mer)):
        p = p * pro_matrix[dic[k_mer[i]]][i]
    return p

In [8]:
def F_given_model(temp_k_mers,background_p):
    '''
    计算F(计算F 若更新使F变大，更新；若不能变大，不更新)
    temp_k_mers : 
    [['A', 'C', 'G', 'T', 'T', 'A', 'C', 'T'],
     ['T', 'G', 'G', 'G', 'C', 'C', 'A', 'A'],
     ['T', 'G', 'A', 'C', 'A', 'T', 'G', 'A'],
     ['C', 'C', 'C', 'G', 'T', 'G', 'A', 'C'],
     ['G', 'A', 'T', 'C', 'C', 'A', 'C', 'A'],
     ['G', 'G', 'T', 'T', 'G', 'T', 'T', 'A'],
     ['T', 'G', 'G', 'T', 'G', 'T', 'C', 'A'],
     ['G', 'G', 'T', 'G', 'G', 'G', 'T', 'T'],
     ['A', 'T', 'T', 'C', 'C', 'G', 'C', 'A'],
     ['C', 'C', 'G', 'A', 'A', 'T', 'A', 'T']]
    background_p : [0.2496, 0.2492, 0.2516, 0.2496]
    '''
    dic = {'A':0,'T':1,'C':2,'G':3}
    sigma = ['A','T','C','G']
    
    #motif_count_matrix
    count_matrix = [[0 for i in range(motif_len)] for j in range(4)]
    for i in range(motif_len):
        for j in range(fa_Num):
            if temp_k_mers[j][i] == 'A':
                count_matrix[0][i] += 1
            elif temp_k_mers[j][i] == 'T':
                count_matrix[1][i] += 1
            elif temp_k_mers[j][i] == 'C':
                count_matrix[2][i] += 1
            elif temp_k_mers[j][i] == 'G':
                count_matrix[3][i] += 1
    count_matrix = np.array(count_matrix)
    '''
    [[3, 2, 2, 2, 3, 3, 4, 6],
     [3, 2, 5, 3, 3, 4, 3, 4],
     [3, 4, 2, 4, 4, 2, 4, 2],
     [4, 5, 4, 4, 3, 4, 2, 1]]
    '''
    
    #motif_probability_matrix
    pro_matrix = count_matrix/fa_Num
    '''
    [[0.23076923 0.15384615 0.15384615 0.15384615 0.23076923 0.23076923
      0.30769231 0.46153846]
     [0.23076923 0.15384615 0.38461538 0.23076923 0.23076923 0.30769231
      0.23076923 0.30769231]
     [0.23076923 0.30769231 0.15384615 0.30769231 0.30769231 0.15384615
      0.30769231 0.15384615]
     [0.30769231 0.38461538 0.30769231 0.30769231 0.23076923 0.30769231
      0.15384615 0.07692308]]
    '''
    #print(count_matrix)
    F = 0
    for i in range(len(temp_k_mers[0])):
        for j in range(4):
            if count_matrix[j][i] == 0:
                F += 0
            else:
                F += count_matrix[j][i]*math.log2(pro_matrix[j][i]/background_p[j])
    return F

In [24]:
########################################## 没有F
######### 设初值 
##### step1 随机选初始site
sites = [random.randint(0, (len(fa_Seq[0])-motif_len)) for i in range(fa_Num)] 
k_mers = []
for i in range(fa_Num):
    k_mers.append(list(fa_Seq[i][sites[i]:sites[i]+motif_len]))
f = 0
A = []
B = [[]]
######### 开始循环
iteration_n = 10000
for itera in range(iteration_n):    
    hide_index = random.randint(0,fa_Num-1)#随机隐藏一行
    temp_motif_matrix = k_mers[:hide_index]+k_mers[hide_index+1:] #生成临时motif matrix
    psudo_count_matrix = [[1 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
    for i in range(motif_len):
        for j in range(fa_Num-1):
            if temp_motif_matrix[j][i] == 'A':
                psudo_count_matrix[0][i] += 1
            elif temp_motif_matrix[j][i] == 'T':
                psudo_count_matrix[1][i] += 1
            elif temp_motif_matrix[j][i] == 'C':
                psudo_count_matrix[2][i] += 1
            elif temp_motif_matrix[j][i] == 'G':
                psudo_count_matrix[3][i] += 1
    psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix     
    pro_matrix = psudo_count_matrix/(fa_Num-1+4) #生成pro_matrix
    prob = [] #按概率选择hiden string的一个k_mer 用来更新对应行的 motif matrix
    hiden_string = fa_Seq[hide_index]
    for i in  range(len(fa_Seq[0])-motif_len+1):
        temp_k_mer = hiden_string[i:i+motif_len]
        prob.append(generate_prob(pro_matrix,temp_k_mer))
    # site 暂时更新
    updated_site = np.random.choice(len(fa_Seq[0])-motif_len+1,1,prob)[0]  
    temp_sites = copy.deepcopy(sites)
    temp_sites[hide_index] = updated_site # 更新site(temp_site)
    # k_mers 暂时更新
    temp_k_mers = copy.deepcopy(k_mers)
    temp_k_mers[hide_index] = list(fa_Seq[hide_index][sites[hide_index]:sites[hide_index]+motif_len]) # 新的k_mers(temp_k_mers)
    sites = copy.deepcopy(temp_sites)
    k_mers = copy.deepcopy(temp_k_mers)
    A.append(sites)
    if itera in [iteration_n-4,iteration_n-3,iteration_n-2,iteration_n-1]:
        B.append(k_mers)

In [21]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num-1):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix

2，4，4，4，（13），2

array([[1, 2, 2, 2, 4, 2],
       [4, 2, 1, 2, 0, 3],
       [1, 2, 2, 2, 4, 2],
       [3, 3, 4, 3, 1, 2]])

In [23]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num-1):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix

2，4，（14），3，（34），（34）

array([[2, 2, 3, 2, 1, 1],
       [4, 2, 2, 2, 2, 2],
       [0, 1, 1, 3, 3, 3],
       [3, 4, 3, 2, 3, 3]])

In [25]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num-1):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix
4，4，3，4，3，2

array([[1, 1, 2, 2, 2, 1],
       [3, 0, 1, 2, 2, 4],
       [1, 3, 4, 2, 4, 2],
       [4, 5, 2, 3, 1, 2]])

In [26]:
A

[[337, 324, 197, 39, 404, 119, 249, 63, 310, 395],
 [337, 324, 197, 39, 404, 229, 249, 63, 310, 395],
 [408, 324, 197, 39, 404, 229, 249, 63, 310, 395],
 [408, 324, 197, 208, 404, 229, 249, 63, 310, 395],
 [408, 324, 197, 208, 404, 229, 249, 211, 310, 395],
 [408, 324, 242, 208, 404, 229, 249, 211, 310, 395],
 [408, 324, 242, 208, 404, 229, 388, 211, 310, 395],
 [408, 324, 242, 109, 404, 229, 388, 211, 310, 395],
 [408, 324, 242, 460, 404, 229, 388, 211, 310, 395],
 [408, 324, 242, 460, 404, 229, 388, 211, 439, 395],
 [408, 324, 242, 30, 404, 229, 388, 211, 439, 395],
 [408, 324, 242, 30, 404, 229, 388, 211, 326, 395],
 [408, 324, 242, 30, 169, 229, 388, 211, 326, 395],
 [408, 324, 56, 30, 169, 229, 388, 211, 326, 395],
 [408, 324, 56, 30, 169, 229, 168, 211, 326, 395],
 [408, 324, 387, 30, 169, 229, 168, 211, 326, 395],
 [408, 324, 387, 30, 169, 229, 168, 211, 120, 395],
 [408, 324, 387, 30, 169, 229, 168, 336, 120, 395],
 [408, 324, 387, 30, 169, 229, 168, 336, 120, 429],
 [408, 324,

In [17]:
########################################## 有F
######### 设初值 
##### step1 随机选初始site
sites = [random.randint(0, (len(fa_Seq[0])-motif_len)) for i in range(fa_Num)] 
k_mers = []
for i in range(fa_Num):
    k_mers.append(list(fa_Seq[i][sites[i]:sites[i]+motif_len]))
f = -100
A = []
B = [[]]
######### 开始循环
iteration_n = 1009
for itera in range(iteration_n):    
    hide_index = random.randint(0,fa_Num-1)#随机隐藏一行
    temp_motif_matrix = k_mers[:hide_index]+k_mers[hide_index+1:] #生成临时motif matrix
    psudo_count_matrix = [[1 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
    for i in range(motif_len):
        for j in range(fa_Num-1):
            if temp_motif_matrix[j][i] == 'A':
                psudo_count_matrix[0][i] += 1
            elif temp_motif_matrix[j][i] == 'T':
                psudo_count_matrix[1][i] += 1
            elif temp_motif_matrix[j][i] == 'C':
                psudo_count_matrix[2][i] += 1
            elif temp_motif_matrix[j][i] == 'G':
                psudo_count_matrix[3][i] += 1
    psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix     
    pro_matrix = psudo_count_matrix/(fa_Num-1+4) #生成pro_matrix
    prob = [] #按概率选择hiden string的一个k_mer 用来更新对应行的 motif matrix
    hiden_string = fa_Seq[hide_index]
    
    for i in  range(len(fa_Seq[0])-motif_len+1):
        temp_k_mer = hiden_string[i:i+motif_len]
        prob.append(generate_prob(pro_matrix,temp_k_mer))
    # site 暂时更新
    updated_site = np.random.choice(len(fa_Seq[0])-motif_len+1,1,prob)[0]  
    temp_sites = copy.deepcopy(sites)
    temp_sites[hide_index] = updated_site # 更新site(temp_site)
    # k_mers 暂时更新
    temp_k_mers = copy.deepcopy(k_mers)
    temp_k_mers[hide_index] = list(fa_Seq[hide_index][sites[hide_index]:sites[hide_index]+motif_len]) # 新的k_mers(temp_k_mers)
    
    # 计算 background probability
    dic = {'A':0,'T':1,'C':2,'G':3}
    background_c = [0 for i in range(4)]
    background_c = np.array(background_c)
    for i in fa_Seq:
        for j in i:
            background_c[dic[j]] +=1
    background_p = background_c/sum(background_c)
    temp_F = F_given_model(temp_k_mers,background_p)
    #print(temp_F)
    if temp_F > f:
        sites = copy.deepcopy(temp_sites)
        k_mers = copy.deepcopy(temp_k_mers)
        f = temp_F
    A.append(sites)
    #print(k_mers)
    if itera in [iteration_n-4,iteration_n-3,iteration_n-2,iteration_n-1]:
        print(itera)
        B.append(k_mers)

1005
1006
1007
1008


In [12]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num-1):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix

341444

array([[2, 1, 5, 2, 2, 3],
       [3, 1, 3, 1, 1, 0],
       [4, 1, 0, 1, 1, 2],
       [0, 6, 1, 5, 5, 4]])

In [15]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num-1):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix

42（123）3（34）1

array([[1, 2, 3, 2, 1, 4],
       [1, 5, 3, 2, 2, 1],
       [1, 0, 3, 4, 3, 1],
       [6, 2, 0, 1, 3, 3]])

In [18]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num-1):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix
2343（23）4
341444

array([[2, 1, 1, 1, 2, 1],
       [5, 1, 3, 0, 3, 3],
       [2, 4, 1, 5, 3, 1],
       [0, 3, 4, 3, 1, 4]])

In [39]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num-1):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix

array([[1, 2, 0, 2, 3, 2],
       [4, 2, 1, 3, 2, 1],
       [0, 0, 4, 0, 2, 3],
       [4, 5, 4, 4, 2, 3]])